## Vanilla Embedding based Tweet Classifier

- `data/emb_results1.csv` -> 0.77076

In [1]:
%load_ext nb_black

import collections
import numpy as np
import pandas as pd
import re
from argparse import Namespace
import os

import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F

import utils
from dataset import SequenceTweetDataset
from classifiers import TweetEmbeddingClassifier

/Users/pardeep/anaconda3/envs/nlp-with-pytorch/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


<IPython.core.display.Javascript object>

In [2]:
args = Namespace(
    # Data and Path hyper parameters
    tweets_csv="data/train_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="models/embeddings/",
    # Model hyper parameters
    glove_filepath="../../data/glove.6B.100d.txt",
    use_glove=False,
    embedding_size=100,
    hidden_dim=100,
    num_channels=100,
    # Training hyper parameter
    seed=1337,
    learning_rate=0.001,
    dropout_p=0.1,
    batch_size=128,
    num_epochs=100,
    early_stopping_criteria=5,
    # Runtime option
    cuda=True,
    catch_keyboard_interrupt=True,
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
)

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir, args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir, args.model_state_file)

    print("Expanded filepaths: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))

# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

# Set seed for reproducibility
utils.set_seed_everywhere(args.seed, args.cuda)

# handle dirs
utils.handle_dirs(args.save_dir)

Expanded filepaths: 
	models/embeddings/document_classification/vectorizer.json
	models/embeddings/document_classification/model.pth
Using CUDA: False


<IPython.core.display.Javascript object>

In [3]:
args.use_glove = True

if args.reload_from_files:
    # training from a checkpoint
    dataset = SequenceTweetDataset.load_dataset_and_load_vectorizer(
        args.tweets_csv, args.vectorizer_file
    )
else:
    # create dataset and vectorizer
    dataset = SequenceTweetDataset.load_dataset_and_make_vectorizer(args.tweets_csv)
    dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.get_vectorizer()

# Use GloVe or randomly initialized embeddings
if args.use_glove:
    words = vectorizer.tweet_vocab._token_to_idx.keys()
    embeddings = utils.make_embedding_matrix(
        glove_filepath=args.glove_filepath, words=words
    )
    print("Using pre-trained embeddings")
else:
    print("Not using pre-trained embeddings")
    embeddings = None

classifier = TweetEmbeddingClassifier(
    embedding_size=args.embedding_size,
    num_embeddings=len(vectorizer.tweet_vocab),
    num_channels=args.num_channels,
    hidden_dim=args.hidden_dim,
    output_dim=1,
    dropout_p=args.dropout_p,
    pretrained_embeddings=embeddings,
    padding_idx=0,
)
print(classifier)

Using pre-trained embeddings
TweetEmbeddingClassifier(
  (emb): Embedding(3111, 100, padding_idx=0)
  (convnet): Sequential(
    (0): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (1): ELU(alpha=1.0)
    (2): Conv1d(100, 100, kernel_size=(3,), stride=(2,))
    (3): ELU(alpha=1.0)
    (4): Conv1d(100, 100, kernel_size=(3,), stride=(2,))
    (5): ELU(alpha=1.0)
    (6): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (7): ELU(alpha=1.0)
  )
  (fc1): Linear(in_features=100, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)


/Users/pardeep/playground/NLP-with-PyTorch/projects/kaggle_nlp_with_disaster_tweets/utils.py:315: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(embedding_i)


<IPython.core.display.Javascript object>

In [4]:
classifier = classifier.to(args.device)
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, mode="min", factor=0.5, patience=1
)
train_state = utils.train_model(
    classifier=classifier,
    loss_func=loss_func,
    optimizer=optimizer,
    scheduler=scheduler,
    dataset=dataset,
    args=args,
)
train_state = utils.evaluate_test_split(
    classifier=classifier,
    dataset=dataset,
    loss_func=loss_func,
    args=args,
    train_state=train_state,
)

Training Routine:   0%|          | 0/100 [00:00<?, ?it/s]

split=train:   0%|          | 0/41 [00:00<?, ?it/s]

split=val:   0%|          | 0/8 [00:00<?, ?it/s]

============ Split=train, Size=5331 ============
============ Split=val, Size=1141 ============
--------------- 0th Epoch Stats---------------
Training Loss=0.6367046709467724, Training Accuracy=61.985518292682926
Validation Loss=0.4669395312666893, Validation Accuracy=79.296875.
------------------------------------------------------------
--------------- 10th Epoch Stats---------------
Training Loss=0.1959298821847613, Training Accuracy=92.98780487804879
Validation Loss=0.6810704581439495, Validation Accuracy=77.1484375.
------------------------------------------------------------
--------------- 20th Epoch Stats---------------
Training Loss=0.17105375866337516, Training Accuracy=94.13109756097562
Validation Loss=0.735496610403061, Validation Accuracy=75.9765625.
------------------------------------------------------------
--------------- 30th Epoch Stats---------------
Training Loss=0.17115213322203335, Training Accuracy=94.16920731707317
Validation Loss=0.7313713729381562, Validatio

<IPython.core.display.Javascript object>

In [15]:
def predict_class(classifier, vectorizer, tweet, max_length, decision_threshold=0.5):
    vectorized_tweet = torch.tensor(
        vectorizer.vectorize(tweet, vector_length=max_length)
    )
    result = classifier(vectorized_tweet.unsqueeze(0))
    probability_value = F.sigmoid(result).item()
    predicted_index = 1 if probability_value >= decision_threshold else 0
    return vectorizer.target_vocab.lookup_index(predicted_index)

<IPython.core.display.Javascript object>

In [16]:
test_dataset = pd.read_csv("data/test.csv")
results = []
for id, _, _, tweet in test_dataset.values:
    prediction = predict_class(
        classifier, dataset.get_vectorizer(), tweet, dataset._max_seq_length + 1
    )
    results.append([id, prediction])
submission_df = pd.DataFrame(results, columns=["id", "target"])
submission_df.to_csv("data/emb_results1.csv", index=False)

<IPython.core.display.Javascript object>

## With Full Dataset 

- `data/emb_results2.csv` -> 0.74256

In [18]:
args = Namespace(
    # Data and Path hyper parameters
    tweets_csv="data/train_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="models/embeddings/",
    # Model hyper parameters
    glove_filepath="../../data/glove.6B.100d.txt",
    use_glove=False,
    embedding_size=100,
    hidden_dim=100,
    num_channels=100,
    # Training hyper parameter
    seed=1337,
    learning_rate=0.001,
    dropout_p=0.1,
    batch_size=128,
    num_epochs=100,
    early_stopping_criteria=5,
    # Runtime option
    cuda=True,
    catch_keyboard_interrupt=True,
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
)

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir, args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir, args.model_state_file)

    print("Expanded filepaths: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))

# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

# Set seed for reproducibility
utils.set_seed_everywhere(args.seed, args.cuda)

# handle dirs
utils.handle_dirs(args.save_dir)

args.use_glove = True

if args.reload_from_files:
    # training from a checkpoint
    dataset = SequenceTweetDataset.load_dataset_and_load_vectorizer(
        args.tweets_csv, args.vectorizer_file
    )
else:
    # create dataset and vectorizer
    dataset = SequenceTweetDataset.load_dataset_and_make_vectorizer(
        args.tweets_csv, use_full_dataset=True
    )
    dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.get_vectorizer()

# Use GloVe or randomly initialized embeddings
if args.use_glove:
    words = vectorizer.tweet_vocab._token_to_idx.keys()
    embeddings = utils.make_embedding_matrix(
        glove_filepath=args.glove_filepath, words=words
    )
    print("Using pre-trained embeddings")
else:
    print("Not using pre-trained embeddings")
    embeddings = None

classifier = TweetEmbeddingClassifier(
    embedding_size=args.embedding_size,
    num_embeddings=len(vectorizer.tweet_vocab),
    num_channels=args.num_channels,
    hidden_dim=args.hidden_dim,
    output_dim=1,
    dropout_p=args.dropout_p,
    pretrained_embeddings=embeddings,
    padding_idx=0,
)
print(classifier)

classifier = classifier.to(args.device)
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, mode="min", factor=0.5, patience=1
)
train_state = utils.train_model(
    classifier=classifier,
    loss_func=loss_func,
    optimizer=optimizer,
    scheduler=scheduler,
    dataset=dataset,
    args=args,
)
train_state = utils.evaluate_test_split(
    classifier=classifier,
    dataset=dataset,
    loss_func=loss_func,
    args=args,
    train_state=train_state,
)

test_dataset = pd.read_csv("data/test.csv")
results = []
for id, _, _, tweet in test_dataset.values:
    prediction = predict_class(
        classifier, dataset.get_vectorizer(), tweet, dataset._max_seq_length + 1
    )
    results.append([id, prediction])
submission_df = pd.DataFrame(results, columns=["id", "target"])
submission_df.to_csv("data/emb_results2.csv", index=False)

Expanded filepaths: 
	models/embeddings/vectorizer.json
	models/embeddings/model.pth
Using CUDA: False
Using pre-trained embeddings
TweetEmbeddingClassifier(
  (emb): Embedding(3111, 100, padding_idx=0)
  (convnet): Sequential(
    (0): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (1): ELU(alpha=1.0)
    (2): Conv1d(100, 100, kernel_size=(3,), stride=(2,))
    (3): ELU(alpha=1.0)
    (4): Conv1d(100, 100, kernel_size=(3,), stride=(2,))
    (5): ELU(alpha=1.0)
    (6): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (7): ELU(alpha=1.0)
  )
  (fc1): Linear(in_features=100, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)


Training Routine:   0%|          | 0/100 [00:00<?, ?it/s]

split=train:   0%|          | 0/59 [00:00<?, ?it/s]

split=val:   0%|          | 0/8 [00:00<?, ?it/s]

============ Split=train, Size=7613 ============
============ Split=val, Size=1141 ============
--------------- 0th Epoch Stats---------------
Training Loss=0.5865282734571877, Training Accuracy=67.90254237288136
Validation Loss=0.4292343817651272, Validation Accuracy=81.640625.
------------------------------------------------------------
--------------- 10th Epoch Stats---------------
Training Loss=0.19085996926335966, Training Accuracy=93.29978813559322
Validation Loss=0.16570794209837916, Validation Accuracy=94.7265625.
------------------------------------------------------------
--------------- 20th Epoch Stats---------------
Training Loss=0.0916260357125331, Training Accuracy=96.82203389830508
Validation Loss=0.09555885102599859, Validation Accuracy=96.77734375.
------------------------------------------------------------
--------------- 30th Epoch Stats---------------
Training Loss=0.052477165204236065, Training Accuracy=97.40466101694918
Validation Loss=0.046623325906693935, Val

<IPython.core.display.Javascript object>

## With 500 Epochs and Full Dataset

- `data/emb_results3.csv` -> 0.74256

In [21]:
args = Namespace(
    # Data and Path hyper parameters
    tweets_csv="data/train_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="models/embeddings/",
    # Model hyper parameters
    glove_filepath="../../data/glove.6B.100d.txt",
    use_glove=False,
    embedding_size=100,
    hidden_dim=100,
    num_channels=100,
    # Training hyper parameter
    seed=1337,
    learning_rate=0.001,
    dropout_p=0.1,
    batch_size=128,
    num_epochs=500,
    early_stopping_criteria=5,
    # Runtime option
    cuda=True,
    catch_keyboard_interrupt=True,
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
)

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir, args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir, args.model_state_file)

    print("Expanded filepaths: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))

# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

# Set seed for reproducibility
utils.set_seed_everywhere(args.seed, args.cuda)

# handle dirs
utils.handle_dirs(args.save_dir)

args.use_glove = True

if args.reload_from_files:
    # training from a checkpoint
    dataset = SequenceTweetDataset.load_dataset_and_load_vectorizer(
        args.tweets_csv, args.vectorizer_file
    )
else:
    # create dataset and vectorizer
    dataset = SequenceTweetDataset.load_dataset_and_make_vectorizer(
        args.tweets_csv, use_full_dataset=True
    )
    dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.get_vectorizer()

# Use GloVe or randomly initialized embeddings
if args.use_glove:
    words = vectorizer.tweet_vocab._token_to_idx.keys()
    embeddings = utils.make_embedding_matrix(
        glove_filepath=args.glove_filepath, words=words
    )
    print("Using pre-trained embeddings")
else:
    print("Not using pre-trained embeddings")
    embeddings = None

classifier = TweetEmbeddingClassifier(
    embedding_size=args.embedding_size,
    num_embeddings=len(vectorizer.tweet_vocab),
    num_channels=args.num_channels,
    hidden_dim=args.hidden_dim,
    output_dim=1,
    dropout_p=args.dropout_p,
    pretrained_embeddings=embeddings,
    padding_idx=0,
)
print(classifier)

classifier = classifier.to(args.device)
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, mode="min", factor=0.5, patience=1
)
train_state = utils.train_model(
    classifier=classifier,
    loss_func=loss_func,
    optimizer=optimizer,
    scheduler=scheduler,
    dataset=dataset,
    args=args,
)
train_state = utils.evaluate_test_split(
    classifier=classifier,
    dataset=dataset,
    loss_func=loss_func,
    args=args,
    train_state=train_state,
)

test_dataset = pd.read_csv("data/test.csv")
results = []
for id, _, _, tweet in test_dataset.values:
    prediction = predict_class(
        classifier, dataset.get_vectorizer(), tweet, dataset._max_seq_length + 1
    )
    results.append([id, prediction])
submission_df = pd.DataFrame(results, columns=["id", "target"])
submission_df.to_csv("data/emb_results3.csv", index=False)

Expanded filepaths: 
	models/embeddings/vectorizer.json
	models/embeddings/model.pth
Using CUDA: False
Using pre-trained embeddings
TweetEmbeddingClassifier(
  (emb): Embedding(3111, 100, padding_idx=0)
  (convnet): Sequential(
    (0): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (1): ELU(alpha=1.0)
    (2): Conv1d(100, 100, kernel_size=(3,), stride=(2,))
    (3): ELU(alpha=1.0)
    (4): Conv1d(100, 100, kernel_size=(3,), stride=(2,))
    (5): ELU(alpha=1.0)
    (6): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (7): ELU(alpha=1.0)
  )
  (fc1): Linear(in_features=100, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)


Training Routine:   0%|          | 0/500 [00:00<?, ?it/s]

split=train:   0%|          | 0/59 [00:00<?, ?it/s]

split=val:   0%|          | 0/8 [00:00<?, ?it/s]

============ Split=train, Size=7613 ============
============ Split=val, Size=1141 ============
--------------- 0th Epoch Stats---------------
Training Loss=0.5865282734571877, Training Accuracy=67.90254237288136
Validation Loss=0.4292343817651272, Validation Accuracy=81.640625.
------------------------------------------------------------
--------------- 10th Epoch Stats---------------
Training Loss=0.19085996926335966, Training Accuracy=93.29978813559322
Validation Loss=0.16570794209837916, Validation Accuracy=94.7265625.
------------------------------------------------------------
--------------- 20th Epoch Stats---------------
Training Loss=0.0916260357125331, Training Accuracy=96.82203389830508
Validation Loss=0.09555885102599859, Validation Accuracy=96.77734375.
------------------------------------------------------------
--------------- 30th Epoch Stats---------------
Training Loss=0.052477165204236065, Training Accuracy=97.40466101694918
Validation Loss=0.046623325906693935, Val

--------------- 330th Epoch Stats---------------
Training Loss=0.037026325856351246, Training Accuracy=98.13294491525423
Validation Loss=0.035025316290557384, Validation Accuracy=98.53515625.
------------------------------------------------------------
--------------- 340th Epoch Stats---------------
Training Loss=0.03635038348640931, Training Accuracy=98.1329449152542
Validation Loss=0.03445377014577389, Validation Accuracy=98.33984374999999.
------------------------------------------------------------
--------------- 350th Epoch Stats---------------
Training Loss=0.03532869914957023, Training Accuracy=98.3448093220339
Validation Loss=0.03610576951177791, Validation Accuracy=98.2421875.
------------------------------------------------------------
--------------- 360th Epoch Stats---------------
Training Loss=0.03592650601008939, Training Accuracy=98.23887711864406
Validation Loss=0.03634186019189656, Validation Accuracy=98.6328125.
-----------------------------------------------------

<IPython.core.display.Javascript object>

## With 500 Epochs and Splitted Dataset

- `data/emb_results4.csv` -> 0.76647

In [23]:
args = Namespace(
    # Data and Path hyper parameters
    tweets_csv="data/train_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="models/embeddings/",
    # Model hyper parameters
    glove_filepath="../../data/glove.6B.100d.txt",
    use_glove=False,
    embedding_size=100,
    hidden_dim=100,
    num_channels=100,
    # Training hyper parameter
    seed=1337,
    learning_rate=0.001,
    dropout_p=0.1,
    batch_size=128,
    num_epochs=500,
    early_stopping_criteria=5,
    # Runtime option
    cuda=True,
    catch_keyboard_interrupt=True,
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
)

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir, args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir, args.model_state_file)

    print("Expanded filepaths: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))

# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

# Set seed for reproducibility
utils.set_seed_everywhere(args.seed, args.cuda)

# handle dirs
utils.handle_dirs(args.save_dir)

args.use_glove = True

if args.reload_from_files:
    # training from a checkpoint
    dataset = SequenceTweetDataset.load_dataset_and_load_vectorizer(
        args.tweets_csv, args.vectorizer_file
    )
else:
    # create dataset and vectorizer
    dataset = SequenceTweetDataset.load_dataset_and_make_vectorizer(
        args.tweets_csv, use_full_dataset=False
    )
    dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.get_vectorizer()

# Use GloVe or randomly initialized embeddings
if args.use_glove:
    words = vectorizer.tweet_vocab._token_to_idx.keys()
    embeddings = utils.make_embedding_matrix(
        glove_filepath=args.glove_filepath, words=words
    )
    print("Using pre-trained embeddings")
else:
    print("Not using pre-trained embeddings")
    embeddings = None

classifier = TweetEmbeddingClassifier(
    embedding_size=args.embedding_size,
    num_embeddings=len(vectorizer.tweet_vocab),
    num_channels=args.num_channels,
    hidden_dim=args.hidden_dim,
    output_dim=1,
    dropout_p=args.dropout_p,
    pretrained_embeddings=embeddings,
    padding_idx=0,
)
print(classifier)

classifier = classifier.to(args.device)
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, mode="min", factor=0.5, patience=1
)
train_state = utils.train_model(
    classifier=classifier,
    loss_func=loss_func,
    optimizer=optimizer,
    scheduler=scheduler,
    dataset=dataset,
    args=args,
)
train_state = utils.evaluate_test_split(
    classifier=classifier,
    dataset=dataset,
    loss_func=loss_func,
    args=args,
    train_state=train_state,
)

test_dataset = pd.read_csv("data/test.csv")
results = []
for id, _, _, tweet in test_dataset.values:
    prediction = predict_class(
        classifier, dataset.get_vectorizer(), tweet, dataset._max_seq_length + 1
    )
    results.append([id, prediction])
submission_df = pd.DataFrame(results, columns=["id", "target"])
submission_df.to_csv("data/emb_results4.csv", index=False)

Expanded filepaths: 
	models/embeddings/vectorizer.json
	models/embeddings/model.pth
Using CUDA: False
Using pre-trained embeddings
TweetEmbeddingClassifier(
  (emb): Embedding(3111, 100, padding_idx=0)
  (convnet): Sequential(
    (0): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (1): ELU(alpha=1.0)
    (2): Conv1d(100, 100, kernel_size=(3,), stride=(2,))
    (3): ELU(alpha=1.0)
    (4): Conv1d(100, 100, kernel_size=(3,), stride=(2,))
    (5): ELU(alpha=1.0)
    (6): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (7): ELU(alpha=1.0)
  )
  (fc1): Linear(in_features=100, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)


Training Routine:   0%|          | 0/500 [00:00<?, ?it/s]

split=train:   0%|          | 0/41 [00:00<?, ?it/s]

split=val:   0%|          | 0/8 [00:00<?, ?it/s]

============ Split=train, Size=5331 ============
============ Split=val, Size=1141 ============
--------------- 0th Epoch Stats---------------
Training Loss=0.6367046709467724, Training Accuracy=61.985518292682926
Validation Loss=0.4669395312666893, Validation Accuracy=79.296875.
------------------------------------------------------------
--------------- 10th Epoch Stats---------------
Training Loss=0.1959298821847613, Training Accuracy=92.98780487804879
Validation Loss=0.6810704581439495, Validation Accuracy=77.1484375.
------------------------------------------------------------
--------------- 20th Epoch Stats---------------
Training Loss=0.17105375866337516, Training Accuracy=94.13109756097562
Validation Loss=0.735496610403061, Validation Accuracy=75.9765625.
------------------------------------------------------------
--------------- 30th Epoch Stats---------------
Training Loss=0.17115213322203335, Training Accuracy=94.16920731707317
Validation Loss=0.7313713729381562, Validatio

--------------- 330th Epoch Stats---------------
Training Loss=0.17132065118086046, Training Accuracy=94.32164634146342
Validation Loss=0.7075831219553947, Validation Accuracy=76.953125.
------------------------------------------------------------
--------------- 340th Epoch Stats---------------
Training Loss=0.17175152861490478, Training Accuracy=94.26448170731709
Validation Loss=0.7472766637802124, Validation Accuracy=75.29296875.
------------------------------------------------------------
--------------- 350th Epoch Stats---------------
Training Loss=0.17294556483989812, Training Accuracy=94.43597560975611
Validation Loss=0.7627811431884766, Validation Accuracy=75.68359375.
------------------------------------------------------------
--------------- 360th Epoch Stats---------------
Training Loss=0.17077579785410954, Training Accuracy=94.2263719512195
Validation Loss=0.7461343631148339, Validation Accuracy=76.56250000000001.
----------------------------------------------------------

<IPython.core.display.Javascript object>